# Enhetsregisteret
Norskregistrerte Enheter i enhetsregisteret blir hentet ut med:

- tilhørende kommune,
- næringskode og 
- info om avvikling og konkurs.

Merk: konkursinformasjon ligger kun ute i 3 år for AS og 1 år for ENK

In [12]:
import pandas as pd
import numpy as np
import requests
from typing import Callable, Any, Sequence
import json
import gzip
from datetime import datetime, date

In [2]:
# ENH_BASE = "https://data.brreg.no/enhetsregisteret/api/enheter" # for å gjøre queries
ENH_BASE = "https://data.brreg.no/enhetsregisteret/oppslag/enheter/lastned" # for å hente ned hele datasettet
# Dokumentasjon: https://data.brreg.no/fullmakt/docs/index.html

def gzip_json_to_dataframe(url: str, cols: list, accessor: Callable[[dict], Sequence], predicate: Callable[[dict], bool]) -> pd.DataFrame:
    json_data = json.loads(
        gzip.decompress(
            requests.get(url).content
        )
    )
    # assume json_data is a list of dicts
    itr = (accessor(d) for d in json_data if predicate(d))
    return pd.DataFrame(itr, columns=cols)


In [48]:
cols = [
    'organisasjonsnummer',
    'organisasjonsform',
    'navn',
    'kommune',
    'kommunenummer',
    'stiftelsesdato',
    'konkurs',
    'underAvvikling',
    'underTvangsavviklingEllerTvangsopplosning',
    'naeringskode1',
    'naeringskode2',
    'naeringskode3'
]

MIN_DATE = np.datetime64('1900-01-01')
def enhetsdata(d: dict) -> tuple|None:
    addr = d.get('forretningsadresse') or d.get('postadresse')
    ncode = [d.get(f"naeringskode{i}", {}).get('kode') for i in range(1,4)]
    date_ = np.datetime64(d.get('stiftelsesdato') or d['registreringsdatoEnhetsregisteret'])
    # try:
    return (
        d['organisasjonsnummer'],
        d['organisasjonsform']['kode'],
        d['navn'],
        addr['kommune'], 
        addr['kommunenummer'],
        max(date_, MIN_DATE), # pandas timestamp har begrenset støtte for datoer <= 1677-09-21
        d['konkurs'],
        d['underAvvikling'],
        d['underTvangsavviklingEllerTvangsopplosning'],
        *ncode
    )
    # except (KeyError, TypeError):
    #     print(json.dumps(d, indent=4))
    #     raise

def is_valid(d):
    addr = d.get('forretningsadresse') or d.get('postadresse')
    return (
        addr['landkode']=='NO' and # ikke utenlandsk
        'naeringskode1' in d # minst 1 næringskode 
    )

In [49]:
df = gzip_json_to_dataframe(ENH_BASE, cols, enhetsdata, is_valid)

In [51]:
df.to_parquet('data/enheter.parquet')